In [24]:
import copy

In [25]:
# read data from file txt -> output: (data :string type)
def read_data(url_file):
    data = []
    with open(url_file, encoding='utf8') as f:
        for line in f:
            data.append(line.strip())
        f.close()
        return data
    return None


# formart data from (data :string type) -> output (states : list, sigma : list, s0: character, end_states: list, transition: list)
def format_data(url_file):
    data = read_data(url_file)

    states = list(data[0].split(","))
    sigma = list(data[1].split(","))
    s0 = list(data[2].split(","))
    end_states = list(data[3].split(","))
    transition = []
    for i in range(4, len(data)):
        a = tuple(data[i].split(","))

        transition.append(a)
    return states, sigma, s0, end_states, transition


# states, sigma,s0, end_states, transition = format_data('input.txt')
# formart input from (states, sigma,s0, end_states, transition) to input_otomat
def formart_input(url_file):
    states, sigma, s0, end_states, transition = format_data(url_file)
    states_number = len(states)
    state_trasition_by_zero = []
    state_trasition_by_one = []
    for i in transition:
        if (i[2] == '0'):
            state_trasition_by_zero.append(i[1])
        elif (i[2] == '1'):
            state_trasition_by_one.append(i[1])
        else:
            print("transition " + str(i) + " error !")
            return None
    return states_number, end_states, state_trasition_by_zero, state_trasition_by_one


class Otomat:
    def __init__(self, states, final_states, by_zero, by_one):
        self.states = states
        self.final_states = final_states
        self.by_zero = by_zero
        self.by_one = by_one

def get_equivalence_classes(automaton):
    equivalence_classes = []
    marked_states = []
    for i in range(0, len(automaton.states)):
        if automaton.states[i] in automaton.final_states:
            marked_states.append((1, (None, None)))  # (<class>, (<transition_by_zero_class>, <transition_by_one_class>))
        else:
            marked_states.append((0, (None, None)))

    old_classes_count = len(set(marked_states))

    while True:
        for state, state_marks in enumerate(marked_states):
            marked_states[state] = \
                (state_marks[0],
                 (marked_states[automaton.by_zero[state]][0], marked_states[automaton.by_one[state]][0]))

        unique_marks = list(set(marked_states))
        for state, state_marks in enumerate(marked_states):
            marked_states[state] = (unique_marks.index(state_marks), (None, None))

        if len(unique_marks) == old_classes_count:
            for equivalence_class in range(len(unique_marks)):
                equivalence_classes.append([state for state, state_marks in enumerate(marked_states)
                                           if state_marks[0] == equivalence_class])
            break

        old_classes_count = len(unique_marks)

    return equivalence_classes


def minimize_automaton(automaton, equivalence_classes):
    minimal_automaton = copy.deepcopy(automaton)

    for equivalence_class in equivalence_classes:
        for state in equivalence_class:
            if minimal_automaton.by_zero[state] in equivalence_class:
                minimal_automaton.by_zero[state] = state
            else:
                minimal_automaton.by_zero[state] = \
                    min(equivalence_classes[[equivalence_classes.index(equivalence_class)
                                             for equivalence_class in equivalence_classes
                                             if minimal_automaton.by_zero[state] in equivalence_class][0]])
            if minimal_automaton.by_one[state] in equivalence_class:
                minimal_automaton.by_one[state] = state
            else:
                minimal_automaton.by_one[state] = \
                    min(equivalence_classes[[equivalence_classes.index(equivalence_class)
                                             for equivalence_class in equivalence_classes
                                             if minimal_automaton.by_one[state] in equivalence_class][0]])

    for equivalence_class in equivalence_classes:
        for state in equivalence_class[1:]:
            minimal_automaton.by_zero[state] = -1
            minimal_automaton.by_one[state] = -1
            if state in minimal_automaton.final_states:
                minimal_automaton.final_states.remove(state)
            minimal_automaton.states[minimal_automaton.states.index(state)] = -1

    not_deleted = lambda x: x != -1
    minimal_automaton.states = list(filter(not_deleted, minimal_automaton.states))
    minimal_automaton.by_zero = list(filter(not_deleted, minimal_automaton.by_zero))
    minimal_automaton.by_one = list(filter(not_deleted, minimal_automaton.by_one))

    for i, state in enumerate(minimal_automaton.states):
        if i < len(minimal_automaton.final_states):
            minimal_automaton.final_states[i] = minimal_automaton.states.index(minimal_automaton.final_states[i])
        minimal_automaton.by_zero[i] = minimal_automaton.states.index(minimal_automaton.by_zero[i])
        minimal_automaton.by_one[i] = minimal_automaton.states.index(minimal_automaton.by_one[i])
    minimal_automaton.states = list(range(len(minimal_automaton.states)))

    return minimal_automaton

In [26]:
if __name__ == '__main__':
    url_filetxt = 'input.txt'
    states_number, end_states, state_trasition_by_zero, state_trasition_by_one = formart_input(url_filetxt)
    otomat = Otomat(states_number, end_states, state_trasition_by_zero, state_trasition_by_one)
    equivalence_classes = get_equivalence_classes(otomat)
    otomat_minimization = minimize_automaton(otomat, equivalence_classes)
    print(otomat_minimization)



TypeError: object of type 'int' has no len()